# Compute metrics on data using stratification 

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os

In [ ]:
temperature_data = pd.read_csv('simulation_results/clean_data/all_temperatures.csv')

In [ ]:
plt.hist(temperature_data['temp'], bins=200)

In [ ]:

# divide in quantiles
temperature_data['quantile'] = pd.qcut(temperature_data['temp'], 
                                        q=11)

quantile_bins = temperature_data['quantile'].cat.categories



In [ ]:
from src.data_loader import load_data
from src.metrics import *

results_pid_nominal = load_data('pid', 'nominal')
results_pid_with_noise = load_data('pid', 'noise')
results_pid_with_disturbances = load_data('pid', 'disturbances')

results_onoff_nominal = load_data('onoff', 'nominal')
results_onoff_with_noise = load_data('onoff', 'noise')
results_onoff_with_disturbances = load_data('onoff', 'disturbances')

results_fuzzy_nominal = load_data('fuzzy', 'nominal')
results_fuzzy_with_noise = load_data('fuzzy', 'noise')
results_fuzzy_with_disturbances = load_data('fuzzy', 'disturbances')

metrics = [steady_state_error, mean_square_error, overshoot, comfort_time, 
            energy_consumed]

results = {
    "results_pid_nominal": results_pid_nominal,
    "results_pid_with_noise": results_pid_with_noise,
    "results_pid_with_disturbances": results_pid_with_disturbances,
     "results_onoff_nominal": results_onoff_nominal,
     "results_onoff_with_noise": results_onoff_with_noise,
     "results_onoff_with_disturbances": results_onoff_with_disturbances,
     "results_fuzzy_nominal": results_fuzzy_nominal,
     "results_fuzzy_with_noise": results_fuzzy_with_noise,
     "results_fuzzy_with_disturbances": results_fuzzy_with_disturbances,
}

In [ ]:
plt.hist(temperature_data['temp'], bins=100, alpha=0.5, label='All Temperatures', density=True)
outsideTemp =  [df['outsideTemp'].iloc[0] for df in results_pid_nominal]
#outsideTemp = pd.concat(outsideTemp, ignore_index=True)
plt.hist(outsideTemp, bins=50, alpha=0.5, label='PID Nominal', density=True)

plt.legend()

In [ ]:


def precompute_metric_values(scenario_results, metric_fn):
    """Precompute the metric value for each simulation"""
    return [
        {
            'df': df,
            'initial_temp': df['outsideTemp'].iloc[0],
            'value': metric_fn(df, 'temperatureSensor_T', 20.0, 'windowState')
        }
        for df in scenario_results
    ]


def stratify(precomputed, quantile_bins):
    """Stratify precomputed results by initial temperature"""
    temps = [entry['initial_temp'] for entry in precomputed]
    labels = pd.cut(temps, bins=quantile_bins)
    strata = {}

    for entry, label in zip(precomputed, labels):
        strata.setdefault(label, []).append(entry['value'])

    return strata


def stratified_bootstrap_ci(precomputed, quantile_bins, num_samples=1000, confidence=0.95):
    strata = stratify(precomputed, quantile_bins)
    combined_means = []

    for _ in range(num_samples):
        print(f"Bootstrap iteration {_ + 1}/{num_samples}")
        stratum_means = []
        for values in strata.values():
            if not values:
                continue
            resampled = random.choices(values, k=len(values))
            stratum_means.append(np.mean(resampled))
        combined_means.append(np.mean(stratum_means))

    lower = np.percentile(combined_means, (1 - confidence) / 2 * 100)
    upper = np.percentile(combined_means, (1 + confidence) / 2 * 100)
    mean = np.mean(combined_means)
    return mean, lower, upper


def stratified_bootstrap_variance_ci(precomputed, quantile_bins, num_samples=1000, confidence=0.95):
    strata = stratify(precomputed, quantile_bins)
    combined_variances = []

    for _ in range(num_samples):
        stratum_means = []
        for values in strata.values():
            if not values:
                continue
            resampled = random.choices(values, k=len(values))
            stratum_means.append(np.mean(resampled))
        combined_variances.append(np.var(stratum_means, ddof=1))

    lower = np.percentile(combined_variances, (1 - confidence) / 2 * 100)
    upper = np.percentile(combined_variances, (1 + confidence) / 2 * 100)
    var = np.mean(combined_variances)
    return var, lower, upper


def calculate_and_export_stats(scenario_results, metrics, scenario_name):
    aggregated_stats = []
    long_format_records = []

    for metric in metrics:
        # 1. Precompute values only once
        precomputed = precompute_metric_values(scenario_results, metric)

        # 2. Confidence intervals using precomputed values
        mean, mean_ci_lower, mean_ci_upper = stratified_bootstrap_ci(precomputed, quantile_bins)
        var, var_ci_lower, var_ci_upper = stratified_bootstrap_variance_ci(precomputed, quantile_bins)

        # 3. Store overall results
        aggregated_stats.append({
            'metric': metric.__name__,
            'mean': mean,
            'mean_ci_lower': mean_ci_lower,
            'mean_ci_upper': mean_ci_upper,
            'var': var,
            'var_ci_lower': var_ci_lower,
            'var_ci_upper': var_ci_upper
        })

        # 4. Export per-simulation values
        for i, entry in enumerate(precomputed):
            long_format_records.append({
                'controller': scenario_name,
                'run': i,
                'metric': metric.__name__,
                'value': entry['value']
            })

    # Export results
    pd.DataFrame(aggregated_stats).to_csv(f"simulation_results/statistics/{scenario_name}_aggregated_low_var.csv", index=False)
    pd.DataFrame(long_format_records).to_csv(f"simulation_results/statistics/{scenario_name}_all_low_var.csv", index=False)


In [ ]:
for name, scenario in results.items():
    calculate_and_export_stats(scenario, metrics, scenario_name=name)
    print(f"Stats for {name} calculated and exported.")


In [ ]:


def compare_and_plot(original_file, stratified_file, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # === LOAD AND CLEAN DATA ===
    orig = pd.read_csv(original_file).set_index("metric").dropna()
    strat = pd.read_csv(stratified_file).set_index("metric").dropna()

    common_metrics = orig.index.intersection(strat.index)
    if common_metrics.empty:
        print(f"No common metrics in: {original_file}")
        return

    # === CALCULATE VARIANCE REDUCTION ===
    comparison_df = pd.DataFrame(index=common_metrics)
    comparison_df["original_var"] = orig.loc[common_metrics, "var"]
    comparison_df["stratified_var"] = strat.loc[common_metrics, "var"]
    comparison_df["reduction_%"] = 100 * (
        comparison_df["original_var"] - comparison_df["stratified_var"]
    ) / comparison_df["original_var"]

    # === ADD MEAN ± CI INFO ===
    for prefix, df in [("original", orig), ("stratified", strat)]:
        comparison_df[f"{prefix}_mean"] = df.loc[common_metrics, "mean"]
        comparison_df[f"{prefix}_ci_low"] = df.loc[common_metrics, "mean_ci_lower"]
        comparison_df[f"{prefix}_ci_up"] = df.loc[common_metrics, "mean_ci_upper"]

    # === PRINT REDUCTION TABLE ===
    print(f"\n📊 Variance reduction for: {os.path.basename(original_file)}")
    print(comparison_df[["original_var", "stratified_var", "reduction_%"]].round(4))

    # === PLOT EACH METRIC ===
    for metric in common_metrics:
        fig, ax = plt.subplots(figsize=(6, 4))
        data = comparison_df.loc[metric]

        # Mean and CI values
        orig_mean = data["original_mean"]
        strat_mean = data["stratified_mean"]
        orig_err = [
            orig_mean - data["original_ci_low"],
            data["original_ci_up"] - orig_mean
        ]
        strat_err = [
            strat_mean - data["stratified_ci_low"],
            data["stratified_ci_up"] - strat_mean
        ]

        # Plotting
        ax.errorbar(0, orig_mean, yerr=[[orig_err[0]], [orig_err[1]]],
                    fmt='o', label="Original", capsize=5)
        ax.errorbar(1, strat_mean, yerr=[[strat_err[0]], [strat_err[1]]],
                    fmt='s', label="Stratified", capsize=5)

        ax.set_xticks([0, 1])
        ax.set_xticklabels(["Original", "Stratified"])
        ax.set_title(f"Mean ± CI: {metric}")
        ax.set_ylabel("Value")
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
    

    

In [ ]:
results_files = [
    "simulation_results/statistics/results_pid_nominal_aggregated.csv",
    "simulation_results/statistics/results_pid_with_noise_aggregated.csv",
    "simulation_results/statistics/results_pid_with_disturbances_aggregated.csv",
    "simulation_results/statistics/results_onoff_nominal_aggregated.csv",
    "simulation_results/statistics/results_onoff_with_noise_aggregated.csv",
    "simulation_results/statistics/results_onoff_with_disturbances_aggregated.csv",
    "simulation_results/statistics/results_fuzzy_nominal_aggregated.csv",
    "simulation_results/statistics/results_fuzzy_with_noise_aggregated.csv",
    "simulation_results/statistics/results_fuzzy_with_disturbances_aggregated.csv"
]

results_files_stratified = [
    "simulation_results/statistics/results_pid_nominal_aggregated_low_var.csv",
    "simulation_results/statistics/results_pid_with_noise_aggregated_low_var.csv",
    "simulation_results/statistics/results_pid_with_disturbances_aggregated_low_var.csv",
    "simulation_results/statistics/results_onoff_nominal_aggregated_low_var.csv",
    "simulation_results/statistics/results_onoff_with_noise_aggregated_low_var.csv",
    "simulation_results/statistics/results_onoff_with_disturbances_aggregated_low_var.csv",
    "simulation_results/statistics/results_fuzzy_nominal_aggregated_low_var.csv",
    "simulation_results/statistics/results_fuzzy_with_noise_aggregated_low_var.csv",
    "simulation_results/statistics/results_fuzzy_with_disturbances_aggregated_low_var.csv"
]

for original, stratified in zip(results_files, results_files_stratified):
    # Derive output directory name from file prefix
    base_name = os.path.basename(original).replace("_aggregated.csv", "")
    output_dir = f"plots/{base_name}"
    compare_and_plot(original, stratified, output_dir)
